In [1]:
import imageio
import os
import pandas as pd
import numpy as np
from skimage.transform import resize
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util

In [2]:
folders = os.listdir('train')

In [3]:
class_label = {}
index = 0
for foldername in folders:
    class_label[foldername] = index
    index += 1
class_label

{'Black-grass': 0,
 'Charlock': 1,
 'Cleavers': 2,
 'Common Chickweed': 3,
 'Common wheat': 4,
 'Fat Hen': 5,
 'Loose Silky-bent': 6,
 'Maize': 7,
 'Scentless Mayweed': 8,
 'Shepherds Purse': 9,
 'Small-flowered Cranesbill': 10,
 'Sugar beet': 11}

In [4]:
def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)

def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]

available_transformations = {
    'rotate': random_rotation,
    #'noise': random_noise,
    #'horizontal_flip': horizontal_flip
}

In [5]:
img_df = {'image': [],'label': []}

for foldername in folders:
    plantname = foldername
    foldername = os.path.join('train', foldername)
    for filename in os.listdir(foldername):
        filename = os.path.join(foldername, filename)
        im = imageio.imread(filename)
        img = resize(im, (64, 64, 1))
        img_df['image'].append(img)
        img_df['label'].append(class_label[plantname])
        
        num_transformations_to_apply = random.randint(1, len(available_transformations))
        num_transformations = 0
        transformed_image = None
        while num_transformations <= num_transformations_to_apply:
            # choose a random transformation to apply for a single image
            key = random.choice(list(available_transformations))
            transformed_image = available_transformations[key](img)
            num_transformations += 1
            img_df['image'].append(transformed_image)
            img_df['label'].append(class_label[plantname])

C:\AIML\Anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\AIML\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [6]:
print('Classes', np.unique(img_df['label']))

Classes [ 0  1  2  3  4  5  6  7  8  9 10 11]


In [7]:
print('No of Images', len(img_df['image']))

No of Images 14250


In [8]:
X = np.array(img_df['image'])
Y = img_df['label']

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=1)

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
tf.reset_default_graph()

Using TensorFlow backend.


In [12]:
from keras.preprocessing.image import ImageDataGenerator

# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=360,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.3,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.3,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

In [13]:
# Define model
model = Sequential()

# 1st Conv Layer
model.add(Conv2D(64, (3, 3), input_shape=(64, 64, 1)))
model.add(Activation('relu'))

# 2nd Conv Layer
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

# Fully Connected Layer
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))

#Batch Normalisation
model.add(keras.layers.BatchNormalization())

# Prediction Layer
model.add(Dense(output_dim=12, init='he_normal', bias=True))
model.add(Activation('softmax'))

# Loss and Optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
# Store Training Results
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]

C:\AIML\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=12, kernel_initializer="he_normal", use_bias=True)`


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 64)        640       
_________________________________________________________________
activation_1 (Activation)    (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 60, 60, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               29491328  
_________________________________________________________________
activation_3 (Activation)    (None, 128)               0         
__________

In [ ]:
#model.fit(x_train, y_train, batch_size=100, epochs=10,
#           validation_data=(x_test, y_test), callbacks=callback_list)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),
               steps_per_epoch=10*len(x_train)/100,
               epochs=10,
               verbose=1,
               shuffle=True,
               validation_data=(x_test, y_test),
               callbacks=callback_list)

Epoch 1/10
998/997 [==============================] - 2457s 2s/step - loss: 2.2259 - acc: 0.2294 - val_loss: 2.1294 - val_acc: 0.2737
Epoch 2/10
998/997 [==============================] - 2313s 2s/step - loss: 2.0637 - acc: 0.2999 - val_loss: 2.4829 - val_acc: 0.2014
Epoch 3/10
998/997 [==============================] - 2300s 2s/step - loss: 1.9788 - acc: 0.3303 - val_loss: 3.0575 - val_acc: 0.1544
Epoch 4/10
998/997 [==============================] - 2294s 2s/step - loss: 1.9263 - acc: 0.3448 - val_loss: 7.0386 - val_acc: 0.0552
Epoch 5/10
998/997 [==============================] - 2293s 2s/step - loss: 1.8713 - acc: 0.3615 - val_loss: 1.8999 - val_acc: 0.3609
Epoch 6/10
998/997 [==============================] - 2255s 2s/step - loss: 1.7427 - acc: 0.3952 - val_loss: 2.6684 - val_acc: 0.1965
Epoch 7/10
998/997 [==============================] - 2221s 2s/step - loss: 1.6429 - acc: 0.4257 - val_loss: 1.7651 - val_acc: 0.3911
Epoch 8/10
826/997 [=======================>......] - ETA: 7:4

In [15]:
y_pred = model.predict(x_test)

In [16]:
y_pred_lst = np.argmax(y_pred, axis=1)
y_pred_df = pd.DataFrame(y_pred_lst)
y_pred_df.head(5)

,0
0,6
1,5
2,0
3,5
4,1


In [17]:
from sklearn.metrics import classification_report  
print(classification_report(y_test, y_pred_df))

              precision    recall  f1-score   support

           0       0.67      0.55      0.60       212
           1       0.67      0.57      0.61       292
           2       0.57      0.45      0.50       234
           3       0.59      0.60      0.59       546
           4       0.54      0.43      0.48       157
           5       0.52      0.53      0.52       390
           6       0.47      0.69      0.56       500
           7       0.73      0.54      0.62       165
           8       0.52      0.60      0.56       396
           9       0.45      0.32      0.37       174
          10       0.67      0.66      0.66       417
          11       0.57      0.49      0.53       317

   micro avg       0.56      0.56      0.56      3800
   macro avg       0.58      0.53      0.55      3800
weighted avg       0.57      0.56      0.56      3800

